In [ ]:
!pip install -q transformers datasets peft accelerate trl bitsandbytes


In [ ]:
!pip uninstall -y trl
!pip install git+https://github.com/huggingface/trl.git


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=True,
    device_map="auto",
    load_in_4bit=True
)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

df = pd.read_json("/content/CLEANED_FINAL_INSTRUCTION_DATA.jsonl", lines=True)
df.head()


In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df[["prompt", "response"]])
hf_dataset


In [ ]:
def format_and_tokenize(example):
    formatted = f"### Prompt:\n{example['prompt']}\n\n### Response:\n{example['response']}"
    return tokenizer(formatted, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = hf_dataset.map(format_and_tokenize)
tokenized_dataset


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./nightline-llama7b-instruct",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=20,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
    push_to_hub=False
)


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("nightline-lora-adapter")
tokenizer.save_pretrained("nightline-lora-adapter")


In [ ]:
!zip -r nightline-lora-adapter.zip nightline-lora-adapter
